# Graph


Graph Terminologies #

- Degree of a Vertex: The total number of edges incident on a vertex. There are two types of degrees:
    - In-Degree: The total number of incoming edges of a vertex.
    - Out-Degree: The total number of outgoing edges of a vertex.
    
- Parallel Edges: Two undirected edges are parallel if they have the same end vertices. Two directed edges are parallel if they have the same starting and ending vertices.

- Self Loop: This occurs when an edge starts and ends on the same vertex.

- Adjacency: Two vertices are said to be adjacent if there is an edge connecting them directly.

Both representations are suitable for different situations. If your application frequently manipulates vertices, the adjacency list is a better choice.

If you are dealing primarily with edges, the adjacency matrix is the more efficient approach.

## Challenge 1: Implement Breadth First Search

You have to implement the Breadth First Search traversal in Python. We have already covered the logic behind this algorithm. All that’s left to do is to flesh it out in code.

To solve this problem, all the previously implemented data structures will be available to us.

Input #
A directed graph in the form of an adjacency list and an integer indicating the starting vertex number (source).

Output #
A string containing the vertices of the graph listed in the correct order of traversal.

## Challenge 3: Detect Cycle in a Directed Graph


The concept of loops or cycles is very common in graph theory. A cycle exists when you traverse the directed graph and come upon a vertex that has already been visited.

You have to implement the detect_cycle function which tells you whether or not a graph contains a cycle.

Input #
A directed graph.

Output #
True if a cycle exists. False if it doesn’t.

In [2]:
from Graph import Graph
# We only need Graph and Stack for this Challenge!

def detect_cycle(g):
    # visited list to keep track of the nodes that have been visited
    # since the beginning of the algorithm
    visited = [False] * g.vertices

    # rec_node_stack keeps track of the nodes which are part of the
    # current recursive call
    rec_node_stack = [False] * g.vertices

    for node in range(g.vertices):
        # DFS recursion call
        if detect_cycle_rec(g, node, visited, rec_node_stack):
            return True

    return False


def detect_cycle_rec(g, node, visited, rec_node_stack):
    # Node was already in recursion stack. Cycle found.
    if (rec_node_stack[node]):
        return True

    # It has been visited before this recursion
    if (visited[node]):
        return False
    # Mark current node as visited and
    # add to recursion stack
    visited[node] = True
    rec_node_stack[node] = True

    head_node = g.array[node].head_node
    while(head_node is not None):
        # Pick adjacent node and call it recursively
        adjacent = head_node.data
        # If the node is visited again in the same recursion => Cycle found
        if (detect_cycle_rec(g, adjacent, visited, rec_node_stack)):
            return True
        head_node = head_node.next_element

    # remove the node from the recursive call
    rec_node_stack[node] = False
    return False


g1 = Graph(4)
g1.add_edge(0, 1)
g1.add_edge(1, 2)
g1.add_edge(1, 3)
g1.add_edge(3, 0)
g2 = Graph(3)
g2.add_edge(0, 1)
g2.add_edge(1, 2)

print(detect_cycle(g1))
print(detect_cycle(g2))

True
False


## Find a "Mother Vertex" in a Directed Graph

You have to implement the find_mother_vertex() function which will take a directed graph as an input and find out which vertex is the mother vertex in the graph.

By definition, the mother vertex is a vertex in a graph such that all other vertices in a graph can be reached by following a path from that vertex. A graph can have multiple mother vertices, but you only need to find one.

Input #
A directed graph

Output #
Returns the value of the mother vertex if it exists. Otherwise, it returns -1

### Solution #1: Naive solution

This is the brute force approach for solving this problem. We run a DFS on each vertex using perform_DFS and keep track of the number of vertices visited in the search. If it is equal to g.vertices, then that vertex can reach all the vertices and is, hence, a mother vertex. Since we run a DFS on each node, the time complexity is O(V(V + E))

In [6]:
from Graph import Graph
from Stack import MyStack
# We only need Graph and Stack for this question!


def find_mother_vertex(g):
    # Traverse the Graph Array and perform DFS operation on each vertex
    # The vertex whose DFS Traversal results is equal to the total number
    # of vertices in graph is a mother vertex
    
    for i in range(g.vertices):
        num_of_vertices_reached = perform_DFS(g, i)
        if (num_of_vertices_reached is g.vertices):
            return i
    return - 1

    # Performs DFS Traversal on graph starting from source
    # Returns total number of vertices which can be reached from source


def perform_DFS(g, start_node):
    num_of_vertices = g.vertices
    vertices_reached = 0  # To store how many vertices reached from source
    # A list to hold the history of visited nodes (by default all false)
    # Make a node visited whenever you push it into stack
    visited = [False] * num_of_vertices

    # Create Stack (Implemented in previous section) for Depth First Traversal
    # and Push source in it
    stack = MyStack()
    stack.push(start_node)
    visited[start_node] = True
    
    # Traverse while stack is not empty
    while (stack.is_empty() is False):
        # Pop a vertex/node from stack
        current_node = stack.pop()
        # Get adjacent vertices to the current_node from the list,
        # and if only push unvisited adjacent vertices into stack
        temp = g.array[current_node].head_node
        while (temp is not None):
            if (visited[temp.data] is False):
                stack.push(temp.data)
                visited[temp.data] = True
                vertices_reached += 1
            # 深さ優先探索のため、next_element
            temp = temp.next_element
        # end of while
    return vertices_reached + 1  # +1 is to include the source itself


g = Graph(4)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(3, 0)
g.add_edge(3, 1)
print(find_mother_vertex(g))

3


### Solution #2: Last finished vertex

This solution is based on Kosaraju’s Strongly Connected Component Algorithm. Initially, we run the DFS on the whole graph in a loop (line 16). The DFS ensures that all the nodes in the graph are visited. If the graph is disconnected, the visited list will still have some vertices which haven’t been set to True

The theory is that the last vertex visited in the recursive DFS will be the mother vertex. This is because, at the last vertex, all slots in visited would be True (DFS only stops when all nodes are visited). Hence, we keep track of this last vertex using last_v.

Then, we reset the visited list and run the DFS only on last_v. If it visits all nodes, it is a mother vertex. Otherwise, a mother vertex does not exist. The only limitation in this algorithm is that it can detect one mother vertex, even if others exist.

The DFS of the whole graph works in O(V + E). If a mother vertex exists, the second DFS takes O(V + E) as well. Therefore, the complete time complexity for this algorithm is O(V + E).

In [7]:
from Graph import Graph
from Stack import MyStack
# We only need Graph and Stack for this question!


def find_mother_vertex(g):
    # visited[] is used for DFS. Initially all are
    # initialized as not visited
    visited = [False]*(g.vertices)

    # To store last finished vertex (or mother vertex)
    last_v = 0

    # Do a DFS traversal and find the last finished
    # vertex
    for i in range(g.vertices):
        if visited[i] is False:
            perform_DFS(g, i, visited)
            last_v = i

    # If there exist mother vertex (or vetices) in given
    # graph, then v must be one (or one of them)

    # Now check if v is actually a mother vertex (or graph
    # has a mother vertex). We basically check if every vertex
    # is reachable from v or not.

    # Reset all values in visited[] as false and do
    # DFS beginning from v to check if all vertices are
    # reachable from it or not.
    visited = [False]*(g.vertices)
    perform_DFS(g, last_v, visited)
    if any(i is False for i in visited):
        return -1
    else:
        return last_v


# A recursive function to print DFS starting from v
def perform_DFS(g, node, visited):

    # Mark the current node as visited and print it
    visited[node] = True

    # Recur for all the vertices adjacent to this vertex
    temp = g.array[node].head_node
    while(temp):
        if visited[temp.data] is False:
            perform_DFS(g, temp.data, visited)
        temp = temp.next_element


g = Graph(4)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(3, 0)
g.add_edge(3, 1)
print(find_mother_vertex(g))

3


## Challenge 5: Count Number of Edges in an Undirected Graph

You have to implement the num_edges() function which takes an undirected graph and computes the total number of bidirectional edges. An illustration is also provided for your understanding.

Input #
An undirected graph.

Output #
Returns the number of unique edges in the graph.

In [8]:
from Graph import Graph
# We only need Graph for this Question!


def num_edges(g):
    # For undirected graph, just sum up the size of
    # all the adjacency lists for each vertex
    sum = 0
    for i in range(g.vertices):
        temp = g.array[i].head_node
        while (temp is not None):
            sum += 1
            temp = temp.next_element

    # Half the total sum as it is an undirected graph
    return sum//2


g = Graph(9)
g.add_edge(0, 2)
g.add_edge(0, 5)
g.add_edge(2, 3)
g.add_edge(2, 4)
g.add_edge(5, 3)
g.add_edge(5, 6)
g.add_edge(3, 6)
g.add_edge(6, 7)
g.add_edge(6, 8)
g.add_edge(6, 4)
g.add_edge(7, 8)



g2 = Graph(7)
g2.add_edge(1, 2)
g2.add_edge(1, 3)
g2.add_edge(3, 4)
g2.add_edge(3, 5)
g2.add_edge(2, 5)
g2.add_edge(2, 4)
g2.add_edge(4, 6)
g2.add_edge(4, 5)
g2.add_edge(6, 5)


print(num_edges(g))

print(num_edges(g2))

5
4


In [ ]:
from Graph import Graph
# We only need Graph for this Question!


def num_edges(g):
    # For undirected graph, just sum up the size of
    # all the adjacency lists for each vertex
    return sum([g.array[i].length() for i in range(g.vertices)]) // 2



g = Graph(9)
g.add_edge(0, 2)
g.add_edge(0, 5)
g.add_edge(2, 3)
g.add_edge(2, 4)
g.add_edge(5, 3)
g.add_edge(5, 6)
g.add_edge(3, 6)
g.add_edge(6, 7)
g.add_edge(6, 8)
g.add_edge(6, 4)
g.add_edge(7, 8)



g2 = Graph(7)
g2.add_edge(1, 2)
g2.add_edge(1, 3)
g2.add_edge(3, 4)
g2.add_edge(3, 5)
g2.add_edge(2, 5)
g2.add_edge(2, 4)
g2.add_edge(4, 6)
g2.add_edge(4, 5)
g2.add_edge(6, 5)


print(num_edges(g))

print(num_edges(g2))

### Challenge 6: Check if a Path Exists Between Two Vertices

You have to implement the check_path() function. It takes a source vertex and a destination vertex and tells us whether or not a path exists between the two.

Input # A directed graph, a source value, and a destination value.

Output # Returns True if a path exists from the source to the destination.

In [ ]:
from Graph import Graph
from Queue import MyQueue
# We only need Graph and Queue for this Question!


def check_path(g, source, dest):
    # BFS to check path between source and dest
    # Keep track of visited vertices
    visited = [False]*(g.vertices)

    # Create a queue for BFS
    queue = MyQueue()

    # Enque source and mark it as visited
    queue.enqueue(source)
    visited[source] = True

    # Loop to traverse the whole graph using BFS
    while not(queue.is_empty()):

        node = queue.dequeue()

        # Check if dequeued node is the destination
        if node is dest:
            return True

        # Continue BFS by obtaining first element in linked list
        adjacent = g.array[node].head_node
        while adjacent:
            # enqueue adjacent node if it has not been visited
            if visited[adjacent.data] is False:
                queue.enqueue(adjacent.data)
                visited[adjacent.data] = True
            adjacent = adjacent.next_element

    # Destination was not found in the search
    return False


g1 = Graph(9)
g1.add_edge(0, 2)
g1.add_edge(0, 5)
g1.add_edge(2, 3)
g1.add_edge(2, 4)
g1.add_edge(5, 3)
g1.add_edge(5, 6)
g1.add_edge(3, 6)
g1.add_edge(6, 7)
g1.add_edge(6, 8)
g1.add_edge(6, 4)
g1.add_edge(7, 8)
g2 = Graph(4)
g2.add_edge(0, 1)
g2.add_edge(1, 2)
g2.add_edge(1, 3)
g2.add_edge(2, 3)

print(check_path(g1, 0, 7))
print(check_path(g2, 3, 0))

### Challenge 7: Check if a Given Undirected Graph is Tree or not?

The next section will tackle the tree data structure. For now, here’s the basic difference between a graph and a tree. A graph can only be a tree under two conditions:

- There are no cycles.
- The graph is connected.

```
A graph is connected when there is a path between every pair of vertices. In a connected graph, there are no unreachable vertices. Each vertex must be connected to every other vertex through either a direct edge or a graph traversal.
```


You have to implement is_tree() function which will take a graph as an input and find out if it is a tree.

Input #
An undirected graph.

Output #
Returns True if the given graph is a tree. Otherwise, it returns False.

In [ ]:
from Graph import Graph
# We only need Graph for this Question!


def is_tree(g):
    # All vertices unvisited
    visited = [False] * g.vertices

    # Check cycle using recursion stack
    # Also mark nodes visited to check connectivity
    if check_cycle(g, 0, visited, -1) is True:
        return False

    # Check if all nodes we visited from the source (graph is connected)
    for i in range(len(visited)):
        # Graph is not connected
        if visited[i] is False:
            return False
    # Not cycle and connected graph
    return True


def check_cycle(g, node, visited, parent):
    # Mark node as visited
    visited[node] = True

    # Pick adjacent node and run recursive DFS
    adjacent = g.array[node].head_node
    while adjacent:
        if visited[adjacent.data] is False:
            if check_cycle(g, adjacent.data, visited, node) is True:
                return True

        # If adjacent is visited and not the parent node of the current node
        elif adjacent.data is not parent:
            # Cycle found
            return True
        adjacent = adjacent.next_element

    return False


g = Graph(5)
g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(0, 3)
g.add_edge(3, 4)


print(is_tree(g))

In [24]:
from Graph import Graph
from Queue import MyQueue
# We only need Graph and Queue for this Question!


def find_min(g, a, b):
    num_of_vertices = g.vertices
    # A list to hold the history of visited nodes (by default all false)
    # Make a node visited whenever you enqueue it into queue
    visited = [False] * num_of_vertices

    # For keeping track of distance of current_node from source
    distance = [0] * num_of_vertices

    # Create Queue for Breadth First Traversal and enqueue source in it
    queue = MyQueue()
    queue.enqueue(a)
    visited[a] = True
    # Traverse while queue is not empty
    while (not queue.is_empty()):
        # Dequeue a vertex/node from queue and add it to result
        current_node = queue.dequeue()
        print(current_node)
        print('--------')
        # Get adjacent vertices to the current_node from the list,
        # and if they are not already visited then enqueue them in the Queue
        # and also update their distance from `a`
        # by adding 1 in current_nodes's distance
        
        # g.array[current_node] には current_node から距離1で伸びているノードの linkedlist で入っている
        # temp は、そのうちの一つを選んでいる
        temp = g.array[current_node].head_node
        

        while (temp is not None):
            if (not visited[temp.data]) or (temp.data is b):
                # 幅優先で見つけたnodeをenqueue
                queue.enqueue(temp.data)
                visited[temp.data] = True
                distance[temp.data] = distance[current_node] + 1
                if temp.data is b:
                    return distance[b]
            # ｔemp.next_element で幅優先で探索
            temp = temp.next_element
            
    # end of while
    return -1


g = Graph(7)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(2, 4)
g.add_edge(4, 5)
g.add_edge(2, 5)
g.add_edge(5, 6)
g.add_edge(3, 6)
print(find_min(g, 1, 5))

1
--------
3
--------
2
--------
2


## Challenge 9: Remove Edge

You must implement the remove_edge function which takes a source and a destination as arguments. If an edge exists between the two, it should be deleted.

Input #
A directed graph, a source (integer), and a destination (integer).

Output #
A directed graph with the edge between the source and the destination removed.

In [ ]:
from Graph import Graph
# We only need Graph for this Question!


def remove_edge(graph, source, dest):
    # If empty graph
    if(len(graph.array) is 0):
        return graph
    # check if source valid
    if(source >= len(graph.array) or source < 0):
        return graph
    # check if dest valid
    if(dest >= len(graph.array) or dest < 0):
        return graph
    # Delete by calling delete on head of LinkedList
    # Note: the delete method caters for if the edge does not exist
    graph.array[source].delete(dest)
    return graph


g = Graph(5)
g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(1, 3)
g.add_edge(2, 4)
g.add_edge(4, 0)

g.print_graph()

remove_edge(g, 1, 3)

g.print_graph()